In [ ]:
# 本代码由可视化策略环境自动生成 2021年8月22日10:04
# 本代码单元只能在可视化模式下编辑。您也可以拷贝代码，粘贴到新建的代码单元或者策略，然后修改。


from sklearn.linear_model import LinearRegression
# 行业、市值中性模块
def m20_run_bigquant_run(input_1, input_2, input_3):    
    
    # 1、获取特征数值
    df = input_1.read_df()   
    df = df[df['industry_sw_level1_0']>0]                # 去除没有查出行业的股票
    industry_List = df['industry_sw_level1_0'].unique()          # 所有行业代码
    factors_all=df.columns    #获取因子列表

    # 2、获取用来中性化的因子列表 通常是行业和市值
    factor0 = input_2.read_pickle()                         

    #3、需要做清洗的因子列表 
    factors_need_cal=[k for k in set(factors_all)-set(factor0) if k!='date' and k!='instrument' and k[:2]!='m:' and k!='label']

    #4、缺失值处理 按中信一级行业相同个股的平均值填充
    for fac in factors_need_cal:
        df['fac_mean'] = df[['date']+['industry_sw_level1_0']+[fac]].groupby(['date','industry_sw_level1_0']).transform(np.mean)
        df[fac]=df[fac].fillna(df['fac_mean'])
        del df['fac_mean']

    #5、因子异常值处理  
    #  固定比例法
    #for fac in factors_need_cal:
       # df[fac][df[fac]>df[fac].quantile(0.99)]=df[fac].quantile(0.99)
        #df[fac][df[fac]<df[fac].quantile(0.01)]=df[fac].quantile(0.01)
    #  均值标准差法
    #print(df[factors_need_cal].head())
    #for fac in factors_need_cal:
        #df[fac][df[fac]>df[fac].mean()+3*df[fac].std()]=df[fac].mean()+3*df[fac].std()
        #df[fac][df[fac]<=df[fac].mean()-3*df[fac].std()]=df[fac].mean()-3*df[fac].std()
    #  MAD法
    #print(df[factors_need_cal].head())
    for fac in factors_need_cal:

        median = np.median(list(df[fac]))
        MAD = np.mean(abs(df[fac]) - median)
        df[fac][df[fac]>median+6*MAD] = median+6*MAD  # 剔除偏离中位数6倍以上的数据
        df[fac][df[fac]<median-6*MAD] = median-6*MAD
       
    #计算行业哑变量
    dfTmp = df.copy()  #copy一份用于计算行业哑变量
    for n in range(len(industry_List)):                         # 行业哑变量赋值
        dfTmp['industry_%d' % n] = 0
        dfTmp['industry_%d' % n][df['industry_sw_level1_0']==industry_List[n]]=1
                          
    # 准备线性回归参数
    model0 = LinearRegression()
    X = dfTmp[list('industry_%d' % n for n in range(len(industry_List)))+factor0] #组装行业哑变量列和中性化因子列矩阵
    del X['industry_sw_level1_0'] #删去中性化因子中的行业列
    
    #需要计算中性化的因子列表
    factors_need_cal=[k for k in set(factors_all)-set(factor0) if k!='date' and k!='instrument' and k[:2]!='m:' and k!='label']

    # 逐个特征进行行业市值中性化
    from sklearn.preprocessing import scale
    for fac in factors_need_cal:

            y = df[fac] #获取需要中性化的因子暴露值
            model0.fit(X, y)
            df[fac] = y-model0.predict(X)  # 计算因子暴露相对于行业哑变量和中性化因子回归后的残差
            #df[fac]=(df[fac]-np.mean(df[fac]))/np.std(df[fac])#一种与scale基本等效的处理
            df[fac] = scale(df[fac])
             #对残差取Z-Score标准化将计算后的结果返回给df中的各列，即完成中性化后的结果
    #多重共线性分析
    from sklearn.decomposition import PCA
    import matplotlib.pyplot as plt
    pca = PCA(n_components=len(factors_need_cal))
    pca.fit(df[factors_need_cal])
    var= pca.explained_variance_ratio_ #计算每个因子解释程度
    var1=np.cumsum(np.round(pca.explained_variance_ratio_, decimals=4)*100)#累计解释程度
    plt.plot(var1)
    print(var)
    data_1 = DataSource.write_df(df)
    print(data_1)
    return Outputs(data_1=data_1, data_2=None, data_3=None)

# 后处理函数，可选。输入是主函数的输出，可以在这里对数据做处理，或者返回更友好的outputs数据格式。此函数输出不会被缓存。
def m20_post_run_bigquant_run(outputs):
    return outputs

from sklearn.linear_model import LinearRegression
# 行业、市值中性模块
def m5_run_bigquant_run(input_1, input_2, input_3):    
    
    # 1、获取特征数值
    df = input_1.read_df()   
    df = df[df['industry_sw_level1_0']>0]                # 去除没有查出行业的股票
    industry_List = df['industry_sw_level1_0'].unique()          # 所有行业代码
    factors_all=df.columns    #获取因子列表

    # 2、获取用来中性化的因子列表 通常是行业和市值
    factor0 = input_2.read_pickle()                         

    #3、需要做清洗的因子列表 
    factors_need_cal=[k for k in set(factors_all)-set(factor0) if k!='date' and k!='instrument' and k[:2]!='m:' and k!='label']

    #4、缺失值处理 按中信一级行业相同个股的平均值填充
    for fac in factors_need_cal:
        df['fac_mean'] = df[['date']+['industry_sw_level1_0']+[fac]].groupby(['date','industry_sw_level1_0']).transform(np.mean)
        df[fac]=df[fac].fillna(df['fac_mean'])
        del df['fac_mean']

    #5、因子异常值处理  
    #  固定比例法
    #for fac in factors_need_cal:
       # df[fac][df[fac]>df[fac].quantile(0.99)]=df[fac].quantile(0.99)
        #df[fac][df[fac]<df[fac].quantile(0.01)]=df[fac].quantile(0.01)
    #  均值标准差法
    #print(df[factors_need_cal].head())
    #for fac in factors_need_cal:
        #df[fac][df[fac]>df[fac].mean()+3*df[fac].std()]=df[fac].mean()+3*df[fac].std()
        #df[fac][df[fac]<=df[fac].mean()-3*df[fac].std()]=df[fac].mean()-3*df[fac].std()
    #  MAD法
    #print(df[factors_need_cal].head())
    for fac in factors_need_cal:

        median = np.median(list(df[fac]))
        MAD = np.mean(abs(df[fac]) - median)
        df[fac][df[fac]>median+6*MAD] = median+6*MAD  # 剔除偏离中位数6倍以上的数据
        df[fac][df[fac]<median-6*MAD] = median-6*MAD
             
    #计算行业哑变量
    dfTmp = df.copy()  #copy一份用于计算行业哑变量
    for n in range(len(industry_List)):                         # 行业哑变量赋值
        dfTmp['industry_%d' % n] = 0
        dfTmp['industry_%d' % n][df['industry_sw_level1_0']==industry_List[n]]=1
                          
    # 准备线性回归参数
    model0 = LinearRegression()
    X = dfTmp[list('industry_%d' % n for n in range(len(industry_List)))+factor0] #组装行业哑变量列和中性化因子列矩阵
    del X['industry_sw_level1_0'] #删去中性化因子中的行业列
    
    #需要计算中性化的因子列表
    factors_need_cal=[k for k in set(factors_all)-set(factor0) if k!='date' and k!='instrument' and k[:2]!='m:' and k!='label']

    # 逐个特征进行行业市值中性化
    from sklearn.preprocessing import scale
    for fac in factors_need_cal:

            y = df[fac] #获取需要中性化的因子暴露值
            model0.fit(X, y)
            df[fac] = y-model0.predict(X)  # 计算因子暴露相对于行业哑变量和中性化因子回归后的残差
            #df[fac]=(df[fac]-np.mean(df[fac]))/np.std(df[fac])#一种与scale基本等效的处理
            df[fac] = scale(df[fac])
             #对残差取Z-Score标准化将计算后的结果返回给df中的各列，即完成中性化后的结果
    data_1 = DataSource.write_df(df)
    return Outputs(data_1=data_1, data_2=None, data_3=None)
    
# 后处理函数，可选。输入是主函数的输出，可以在这里对数据做处理，或者返回更友好的outputs数据格式。此函数输出不会被缓存。
def m5_post_run_bigquant_run(outputs):
    return outputs

# 回测引擎：初始化函数，只执行一次
def m19_initialize_bigquant_run(context):
    # 加载预测数据
    context.ranker_prediction = context.options['data'].read_df()

    # 系统已经设置了默认的交易手续费和滑点，要修改手续费可使用如下函数
    context.set_commission(PerOrder(buy_cost=0.0003, sell_cost=0.0013, min_cost=5))
    # 预测数据，通过options传入进来，使用 read_df 函数，加载到内存 (DataFrame)
    # 设置买入的股票数量，这里买入预测股票列表排名靠前的5只
    stock_count = 5
    # 每只的股票的权重，如下的权重分配会使得靠前的股票分配多一点的资金，[0.339160, 0.213986, 0.169580, ..]
    context.stock_weights = T.norm([1 / math.log(i + 2) for i in range(0, stock_count)])
    # 设置每只股票占用的最大资金比例
    context.max_cash_per_instrument = 0.2
    context.options['hold_days'] = 5

# 回测引擎：每日数据处理函数，每天执行一次
def m19_handle_data_bigquant_run(context, data):
    #------------------------START:加入下面if的两行代码到之前到主函数的最前部分-------------------
    # 相隔几天（以5天举例）运行一下handle_data函数
    if context.trading_day_index % 5 != 0:
        return 
    #------------------------END：加上这两句代码在主函数就能实现隔几天运行---------------------
    # 按日期过滤得到今日的预测数据
    ranker_prediction = context.ranker_prediction[
        context.ranker_prediction.date == data.current_dt.strftime('%Y-%m-%d')]

    # 1. 资金分配
    # 平均持仓时间是hold_days，每日都将买入股票，每日预期使用 1/hold_days 的资金
    # 实际操作中，会存在一定的买入误差，所以在前hold_days天，等量使用资金；之后，尽量使用剩余资金（这里设置最多用等量的1.5倍）
    is_staging = context.trading_day_index < context.options['hold_days'] # 是否在建仓期间（前 hold_days 天）
    cash_avg = context.portfolio.portfolio_value / context.options['hold_days']
    cash_for_buy = min(context.portfolio.cash, (1 if is_staging else 1.5) * cash_avg)
    cash_for_sell = cash_avg - (context.portfolio.cash - cash_for_buy)
    positions = {e.symbol: p.amount * p.last_sale_price
                 for e, p in context.perf_tracker.position_tracker.positions.items()}

    # 2. 生成卖出订单：hold_days天之后才开始卖出；对持仓的股票，按机器学习算法预测的排序末位淘汰
    if not is_staging and cash_for_sell > 0:
        equities = {e.symbol: e for e, p in context.perf_tracker.position_tracker.positions.items()}
        instruments = list(reversed(list(ranker_prediction.instrument[ranker_prediction.instrument.apply(
                lambda x: x in equities and not context.has_unfinished_sell_order(equities[x]))])))
        # print('rank order for sell %s' % instruments)
        for instrument in instruments:
            context.order_target(context.symbol(instrument), 0)
            cash_for_sell -= positions[instrument]
            if cash_for_sell <= 0:
                break

    # 3. 生成买入订单：按机器学习算法预测的排序，买入前面的stock_count只股票
    buy_cash_weights = context.stock_weights
    buy_instruments = list(ranker_prediction.instrument[:len(buy_cash_weights)])
    max_cash_per_instrument = context.portfolio.portfolio_value * context.max_cash_per_instrument
    for i, instrument in enumerate(buy_instruments):
        cash = cash_for_buy * buy_cash_weights[i]
        if cash > max_cash_per_instrument - positions.get(instrument, 0):
            # 确保股票持仓量不会超过每次股票最大的占用资金量
            cash = max_cash_per_instrument - positions.get(instrument, 0)
        if cash > 0:
            price = data.current(context.symbol(instrument), 'price')  # 最新价格
            stock_num = np.floor(cash/price/100)*100  # 向下取整
            context.order(context.symbol(instrument), stock_num) # 整百下单


# 回测引擎：准备数据，只执行一次
def m19_prepare_bigquant_run(context):
    pass

# 回测引擎：每个单位时间开始前调用一次，即每日开盘前调用一次。
def m19_before_trading_start_bigquant_run(context, data):
    pass


g = T.Graph({

    'm1': 'M.instruments.v2',
    'm1.start_date': '2014-10-31',
    'm1.end_date': '2016-10-31',
    'm1.market': 'CN_STOCK_A',
    'm1.instrument_list': '',
    'm1.max_count': 0,

    'm2': 'M.advanced_auto_labeler.v2',
    'm2.instruments': T.Graph.OutputPort('m1.data'),
    'm2.label_expr': """# 计算收益：5日经波动率调整后收益率
shift(close, -5) / shift(open, -1)/ std(shift(close, -7) / shift(open, -6),5)

# 极值处理：用1%和99%分位的值做clip
clip(label, all_quantile(label, 0.01), all_quantile(label, 0.99))

# 将分数映射到分类，这里使用50个分类
all_wbins(label, 50)

# 过滤掉一字涨停的情况 (设置label为NaN，在后续处理和训练中会忽略NaN的label)
where(shift(high, -1) == shift(low, -1), NaN, label)""",
    'm2.start_date': '',
    'm2.end_date': '',
    'm2.benchmark': '399102.ZIX',
    'm2.drop_na_label': True,
    'm2.cast_label_int': True,

    'm14': 'M.chinaa_stock_filter.v1',
    'm14.index_constituent_cond': ['中证800'],
    'm14.board_cond': ['全部'],
    'm14.industry_cond': ['全部'],
    'm14.st_cond': ['正常'],
    'm14.delist_cond': ['非退市'],
    'm14.output_left_data': False,

    'm8': 'M.chinaa_stock_filter.v1',
    'm8.index_constituent_cond': ['中证800'],
    'm8.board_cond': ['全部'],
    'm8.industry_cond': ['全部'],
    'm8.st_cond': ['正常'],
    'm8.delist_cond': ['非退市'],
    'm8.output_left_data': False,

    'm11': 'M.instruments.v2',
    'm11.start_date': '2016-11-01',
    'm11.end_date': '2017-04-30',
    'm11.market': 'CN_STOCK_A',
    'm11.instrument_list': '',
    'm11.max_count': 0,

    'm26': 'M.input_features.v1',
    'm26.features': """date
pred_label""",

    'm22': 'M.list_input_feature.v11',
    'm22.feature_list': ['mean(close_0,10)/close_0', 'industry_sw_level1_0', 'market_cap_float_0'],

    'm18': 'M.derived_feature_extractor.v3',
    'm18.input_data': T.Graph.OutputPort('m14.data'),
    'm18.features': T.Graph.OutputPort('m22.feature_list'),
    'm18.date_col': 'date',
    'm18.instrument_col': 'instrument',
    'm18.drop_na': False,
    'm18.remove_extra_columns': True,

    'm28': 'M.dropnan.v2',
    'm28.input_data': T.Graph.OutputPort('m18.data'),

    'm16': 'M.derived_feature_extractor.v3',
    'm16.input_data': T.Graph.OutputPort('m8.data'),
    'm16.features': T.Graph.OutputPort('m22.feature_list'),
    'm16.date_col': 'date',
    'm16.instrument_col': 'instrument',
    'm16.drop_na': False,
    'm16.remove_extra_columns': True,

    'm7': 'M.join.v3',
    'm7.data1': T.Graph.OutputPort('m2.data'),
    'm7.data2': T.Graph.OutputPort('m16.data'),
    'm7.on': 'date,instrument',
    'm7.how': 'inner',
    'm7.sort': False,

    'm27': 'M.dropnan.v2',
    'm27.input_data': T.Graph.OutputPort('m7.data'),

    'm25': 'M.list_input_feature.v11',
    'm25.feature_list': [
'industry_sw_level1_0',
'market_cap_float_0'
],

    'm20': 'M.cached.v3',
    'm20.input_1': T.Graph.OutputPort('m27.data'),
    'm20.input_2': T.Graph.OutputPort('m25.feature_list'),
    'm20.run': m20_run_bigquant_run,
    'm20.post_run': m20_post_run_bigquant_run,
    'm20.input_ports': '',
    'm20.params': '{}',
    'm20.output_ports': '',

    'm15': 'M.dropnan.v2',
    'm15.input_data': T.Graph.OutputPort('m20.data_1'),

    'm5': 'M.cached.v3',
    'm5.input_1': T.Graph.OutputPort('m28.data'),
    'm5.input_2': T.Graph.OutputPort('m25.feature_list'),
    'm5.run': m5_run_bigquant_run,
    'm5.post_run': m5_post_run_bigquant_run,
    'm5.input_ports': '',
    'm5.params': '{}',
    'm5.output_ports': '',

    'm21': 'M.dropnan.v2',
    'm21.input_data': T.Graph.OutputPort('m5.data_1'),

    'm4': 'M.preprocessing_standard_scaler.v1',
    'm4.training_ds': T.Graph.OutputPort('m15.data'),
    'm4.features': T.Graph.OutputPort('m22.feature_list'),
    'm4.predict_ds': T.Graph.OutputPort('m21.data'),
    'm4.with_mean': True,
    'm4.with_std': True,

    'm3': 'M.logistic_regression.v1',
    'm3.training_ds': T.Graph.OutputPort('m4.transform_trainds'),
    'm3.features': T.Graph.OutputPort('m22.feature_list'),
    'm3.predict_ds': T.Graph.OutputPort('m4.transform_predictds'),
    'm3.penalty': 'l2',
    'm3.dual': False,
    'm3.fit_intercept': True,
    'm3.tol': 0.0001,
    'm3.C': 1,
    'm3.key_cols': 'date,instrument',
    'm3.workers': 1,
    'm3.other_train_parameters': {},

    'm17': 'M.sort.v4',
    'm17.input_ds': T.Graph.OutputPort('m3.predictions'),
    'm17.sort_by_ds': T.Graph.OutputPort('m26.data'),
    'm17.sort_by': 'pred_label',
    'm17.group_by': 'date',
    'm17.keep_columns': '--',
    'm17.ascending': False,

    'm19': 'M.trade.v4',
    'm19.instruments': T.Graph.OutputPort('m11.data'),
    'm19.options_data': T.Graph.OutputPort('m17.sorted_data'),
    'm19.start_date': '',
    'm19.end_date': '',
    'm19.initialize': m19_initialize_bigquant_run,
    'm19.handle_data': m19_handle_data_bigquant_run,
    'm19.prepare': m19_prepare_bigquant_run,
    'm19.before_trading_start': m19_before_trading_start_bigquant_run,
    'm19.volume_limit': 0.025,
    'm19.order_price_field_buy': 'open',
    'm19.order_price_field_sell': 'close',
    'm19.capital_base': 1000000,
    'm19.auto_cancel_non_tradable_orders': True,
    'm19.data_frequency': 'daily',
    'm19.price_type': '真实价格',
    'm19.product_type': '股票',
    'm19.plot_charts': True,
    'm19.backtest_only': False,
    'm19.benchmark': '399102.ZIX',
})

# g.run({})


def m9_run_bigquant_run(bq_graph, inputs):
    features =['pe_ttm_0', 'shift(close_0,5)/close_0','mean(close_0,10)/close_0']

    parameters_list = []
    for feature in features:
        feature = [feature]
        feature.append('industry_sw_level1_0')
        feature.append('market_cap_float_0')
        parameters = {'m24.feature_list':feature}
        parameters_list.append({'parameters': parameters})
    
    def run(parameters):
        try:
            print(parameters)
            return g.run(parameters)
        except Exception as e:
            print('ERROR --------', e)
            return None
 
    results = T.parallel_map(run, parameters_list, max_workers=2, remote_run=False, silent=True)

    return results


m9 = M.hyper_run.v1(
    run=m9_run_bigquant_run,
    run_now=True,
    bq_graph=g
)


[2020-03-12 10:17:41.857910] INFO: bigquant: T.parallel_map  开始并行运算..
{'m24.feature_list': ['pe_ttm_0', 'industry_sw_level1_0', 'market_cap_float_0']}
{'m24.feature_list': ['shift(close_0,5)/close_0', 'industry_sw_level1_0', 'market_cap_float_0']}
并行运算进度:   0%|          | 0/3 [00:00<?, ?it/s]

[2020-03-12 10:17:41.864940] INFO: bigquant: instruments.v2 开始运行..

[2020-03-12 10:17:41.865966] INFO: bigquant: instruments.v2 开始运行..

[2020-03-12 10:17:41.899170] INFO: bigquant: 命中缓存

[2020-03-12 10:17:41.900767] INFO: bigquant: instruments.v2 运行完成[0.034779s].

...

[2020-03-12 10:17:43.180407] INFO: bigquant: advanced_auto_labeler.v2 开始运行..

[2020-03-12 10:17:43.703853] INFO: 自动标注(股票): 加载历史数据: 1191303 行

[2020-03-12 10:17:43.704938] INFO: 自动标注(股票): 开始标注 ..

[2020-03-12 10:17:44.794855] INFO: 自动标注(股票): 加载历史数据: 1191303 行

[2020-03-12 10:17:44.796098] INFO: 自动标注(股票): 开始标注 ..

In [ ]:
#超参搜索代码
#print(m17.result.best_params_)
#print(m17.result.best_score_)
#自定义运行-并行计算代码

# 查看所有并行任务的运算结果
m4.result
# 查看第一个并行任务的运算结果
m4.result[0]
# 查看第一个并行任务的预测结果前5条记录
m4.result[0].predictions.read_df().head()
# 查看第一个并行任务的运算结果中m19回测模块的回测曲线
m4.result[0]['m19'].display()

In [ ]:
features =['pe_ttm_0', 'shift(close_0,5)/close_0','mean(close_0,10)/close_0']
for feature in features:
        feature = [feature]
        feature.append('industry_sw_level1_0')
        feature.append('market_cap_float_0')
        print(feature)